<div id="BBox" class="alert alert-danger" style="font-family:courier;color:black;">
        <h1>Handling Transactions in Python - Student Registration System</h1>
        <p>In this guide, we will explore how to handle transactions in a Student Registration System using Python and MySQL. We will understand the concept of transactions, ACID properties, and how to manage concurrent transactions in MySQL.</p>
        <h2>1. Understanding Transactions and ACID Properties</h2>
        <h3>What is a Transaction?</h3>
        <p>A transaction is a sequence of one or more SQL operations executed as a single logical unit of work. Transactions ensure that all related operations are successfully completed or none of them are, keeping the database in a consistent state.</p>
        <h3>Why are Transactions Important?</h3>
        <p>Transactions ensure data consistency, especially in scenarios where multiple operations are involved, such as in a student registration process where data is written to multiple tables. If one operation fails, the entire transaction can be rolled back, preventing partial or inconsistent data from being saved.</p>
        <h3>ACID Properties</h3>
        <p>Transactions must follow the ACID properties to ensure database consistency and reliability:</p>
        <ul>
            <li><strong>Atomicity:</strong> All operations in a transaction must succeed; otherwise, the entire transaction is rolled back.</li>
            <li><strong>Consistency:</strong> Transactions take the database from one consistent state to another.</li>
            <li><strong>Isolation:</strong> Transactions are executed independently of one another.</li>
            <li><strong>Durability:</strong> Once a transaction is committed, it is permanently saved in the database.</li>
        </ul>
        <h2>2. Committing and Rolling Back Transactions</h2>
        <h3>Committing Transactions</h3>
        <p>After all operations in a transaction are executed successfully, the transaction is committed. This saves all changes permanently to the database.</p>
        <h4>Example - Committing a Transaction</h4>
        <pre><code>
import mysql.connector

# Establishing a connection to MySQL
connection = mysql.connector.connect(
    host="localhost",
    user="your_username",
    password="your_password",
    database="your_database"
)

# Creating a cursor object
cursor = connection.cursor()

try:
    # Inserting a new student
    cursor.execute("INSERT INTO students (name, email, age, registration_date) VALUES (%s, %s, %s, CURDATE())", 
                   ("Alice", "alice@example.com", 21))
    
    # Inserting the registration details
    cursor.execute("INSERT INTO registrations (student_id, course_id) VALUES (%s, %s)", 
                   (cursor.lastrowid, 101))  # Assume lastrowid gets the student_id

    # Commit the transaction
    connection.commit()
    print("Transaction committed successfully!")

except mysql.connector.Error as error:
    print(f"Error: {error}")
    connection.rollback()  # Roll back the transaction on error
    print("Transaction rolled back.")

finally:
    # Close the cursor and connection
    cursor.close()
    connection.close()
</code></pre>
        <h3>Rolling Back Transactions</h3>
        <p>If an error occurs during the transaction, it can be rolled back. This undoes all changes made by the transaction and reverts the database to its previous state.</p>
        <h4>Example - Rolling Back a Transaction</h4>
        <pre><code>
import mysql.connector

# Establishing a connection to MySQL
connection = mysql.connector.connect(
    host="localhost",
    user="your_username",
    password="your_password",
    database="your_database"
)

# Creating a cursor object
cursor = connection.cursor()
try:
    # Start transaction
    connection.start_transaction()

    # Step 1: Insert a new student
    cursor.execute("""
        INSERT INTO students (name, email, age, registration_date)
        VALUES (%s, %s, %s, CURDATE())
    """, ("Charlie", "charlie@example.com", 22))
    
    # Get the last inserted student_id
    student_id = cursor.lastrowid

    # Step 2: Enroll the student in a course
    # Assume course_id 9999 does not exist to trigger an error
    cursor.execute("""
        INSERT INTO registrations (student_id, course_id)
        VALUES (%s, %s)
    """, (student_id, 9999))  # This will raise an error

    # If both inserts succeed, commit the transaction
    connection.commit()
    print("Transaction committed successfully!")

except mysql.connector.Error as error:
    # An error occurred, rollback the transaction
    print(f"Error: {error}")
    connection.rollback()  # Roll back the transaction to maintain integrity
    print("Transaction rolled back.")

finally:
    # Close the cursor and connection
    cursor.close()
    connection.close()
</code></pre>
        <h2>3. Managing Concurrent Transactions</h2>
        <h3>What is Concurrent Transaction Management?</h3>
        <p>In a multi-user environment, multiple transactions can occur simultaneously. Managing concurrent transactions is important to avoid conflicts such as dirty reads, non-repeatable reads, and lost updates.</p>
        <h3>Why is it Important?</h3>
        <p>In our Student Registration System, multiple users might try to register students at the same time. Proper handling of concurrent transactions ensures that all data remains consistent and isolated, preventing errors like reading uncommitted changes from another transaction.</p>
        <h3>How it Works</h3>
        <p>MySQL provides various isolation levels to manage concurrency:</p>
        <ul>
            <li><strong>READ UNCOMMITTED:</strong> Lowest isolation level, allows dirty reads.</li>
            <li><strong>READ COMMITTED:</strong> Default level, prevents dirty reads.</li>
            <li><strong>REPEATABLE READ:</strong> Ensures consistent data reads within a transaction.</li>
            <li><strong>SERIALIZABLE:</strong> Highest isolation level, prevents any concurrency issues by locking rows until the transaction is complete.</li>
        </ul>
        <h4>Example - Managing Concurrent Transactions</h4>
        <p>You can set the isolation level in Python as follows:</p>
        <pre><code>
import mysql.connector
import threading

# Function to register a student
def register_student(name, email, age):
    # Establish a connection to the MySQL database
    connection = mysql.connector.connect(
        host="localhost",
        user="your_username",
        password="your_password",
        database="your_database"
    )
    cursor = connection.cursor()
    
    try:
            # Set the isolation level to SERIALIZABLE
            connection.start_transaction(isolation_level='SERIALIZABLE')
            
            # Step 1: Insert a new student
            cursor.execute("""
                INSERT INTO students (name, email, age, registration_date)
                VALUES (%s, %s, %s, CURDATE())
            """, (name, email, age))
    
            # Get the last inserted student_id
            student_id = cursor.lastrowid
    
            # Step 2: Assume registering for a course with course_id 101
            cursor.execute("""
                INSERT INTO registrations (student_id, course_id)
                VALUES (%s, %s)
            """, (student_id, 101))
    
            # Commit the transaction
            connection.commit()
            print(f"Transaction committed successfully for {name}!")
    
    except mysql.connector.Error as error:
            # Rollback the transaction in case of an error
            print(f"Error while registering {name}: {error}")
            connection.rollback()
            print("Transaction rolled back.")
    
    finally:
            # Close the cursor and connection
            cursor.close()
            connection.close()

# Creating threads to simulate concurrent student registrations
thread1 = threading.Thread(target=register_student, args=("Alice", "alice@example.com", 22))
thread2 = threading.Thread(target=register_student, args=("Bob", "bob@example.com", 20))

# Starting the threads
thread1.start()
thread2.start()

# Wait for both threads to finish
thread1.join()
thread2.join()
</code></pre>
    <h3>Expinations</h3>
    <ol><li><strong>Function Definition</strong>: The <code>register_student</code> function handles the registration process for a student. It takes <code>name</code>, <code>email</code>, and <code>age</code> as parameters.</li><li><strong>Database Connection</strong>: Inside the function, a new connection to the MySQL database is established for each thread.</li><li><strong>Setting Isolation Level</strong>: The isolation level is set to <strong>SERIALIZABLE</strong> using <code>connection.start_transaction(isolation_level='SERIALIZABLE')</code>. This ensures that the transactions are executed in isolation.</li><li><strong>Inserting Student</strong>: The first <code>INSERT</code> statement adds a new student to the <code>students</code> table.</li><li><strong>Inserting Registration</strong>: The second <code>INSERT</code> statement adds a registration entry for the newly created student in the <code>registrations</code> table.</li><li><strong>Committing Transaction</strong>: If both operations are successful, the transaction is committed.</li><li><strong>Error Handling</strong>: If an error occurs (e.g., a constraint violation), the error is caught, and the transaction is rolled back.</li><li><strong>Creating Threads</strong>: Two threads are created to simulate concurrent registrations for Alice and Bob.</li><li><strong>Starting and Joining Threads</strong>: The threads are started and then joined to wait for both registrations to complete.</li></ol>
        <h2>Conclusion</h2>
        <p>Handling transactions effectively in Python ensures data consistency, especially in multi-operation workflows like the Student Registration System. Understanding how to commit, roll back, and manage concurrent transactions using ACID properties is essential for reliable database management.</p>
<div>


In [20]:
import os
import mysql.connector

# Set an environment variable
os.environ["DB"] = "localhost;root;Avinash@123;python_workshop"
# # from dotenv import load_dotenv

# load_dotenv()  # Load environment variables from .env file

db_details = os.getenv('DB').split(";")
db_host = db_details[0]
db_user = db_details[1]
db_password = db_details[2]
db_name = db_details[3]

connection = mysql.connector.connect(
    host=db_host,
    user=db_user,
    password=db_password,
    database=db_name
)

In [21]:
# create a cursor object
cursor = connection.cursor()

try:
    # Inserting a new student
    cursor.execute("INSERT INTO new_students (name, email, age, registration_date) VALUES (%s, %s, %s, CURDATE())", 
                   ("Alice", "alice@example.com", 21))


    # Inserting the registration details
    cursor.execute("INSERT INTO registrations (student_id, course_id) VALUES (%s, %s)", 
                   (cursor.lastrowid, 'CS101'))  # Assume lastrowid gets the student_id
    
    # Commit the transaction
    connection.commit()
    print("Transaction committed successfully!")

except mysql.connector.Error as error:
    print(f"Error: {error}")
    connection.rollback()  # Roll back the transaction on error
    print("Transaction rolled back.")


# finally:
#     # Close the cursor and connection
#     cursor.close()
#     connection.close()

Transaction committed successfully!


In [23]:
# Creating a cursor object
cursor = connection.cursor()

try:
    # Start transaction
    connection.start_transaction()

    # Step 1: Insert a new student
    cursor.execute("""
        INSERT INTO new_students (name, email, age, registration_date)
        VALUES (%s, %s, %s, CURDATE())
    """, ("Mayank", "mayank@example.com", 28))
    
    # Get the last inserted student_id
    student_id = cursor.lastrowid
    
    # Step 2: Enroll the student in a course
    # Assume course_id 9999 does not exist to trigger an error
    cursor.execute("""
        INSERT INTO registrations (student_id, course_id)
        VALUES (%s, %s)
    """, (student_id, 'CS200000000001'))  # This will raise an error
    
    # If both inserts succeed, commit the transaction
    connection.commit()
    print("Transaction committed successfully!")

except mysql.connector.Error as error:
    # An error occurred, rollback the transaction
    print(f"Error: {error}")
    connection.rollback()  # Roll back the transaction to maintain integrity
    print("Transaction rolled back.")


# finally:
#     # Close the cursor and connection
#     cursor.close()
#     connection.close()

Error: 1406 (22001): Data too long for column 'course_id' at row 1
Transaction rolled back.


In [28]:
import mysql.connector
import threading

# Function to register a student

def register_student(name, email, age):
    # Establish a connection to the MySQL database
    connection = mysql.connector.connect(
        host="localhost",
        user="root",
        password="Avinash@123",
        database="python_workshop"
    )
    cursor = connection.cursor()

    try:
        # Set the isolation level to SERIALIZABLE
        connection.start_transaction(isolation_level='SERIALIZABLE')
    
        # Step 1: Insert a new student
        cursor.execute("""
            INSERT INTO new_students (name, email, age, registration_date)
            VALUES (%s, %s, %s, CURDATE())
        """, (name, email, age))
    
        # Get the last inserted student_id
        student_id = cursor.lastrowid
    
        # Step 2: Assume registering for a course with course_id 101
        cursor.execute("""
            INSERT INTO registrations (student_id, course_id)
            VALUES (%s, %s)
        """, (student_id, 101))
    
        # Commit the transaction
        connection.commit()
        print(f"Transaction committed successfully for {name}!")
        print()
    
    except mysql.connector.Error as error:
        # Rollback the transaction in case of an error
        print(f"Error while registering {name}: {error}")
        connection.rollback()
        print("Transaction rolled back.")
    
    finally:
        # Close the cursor and connection
        cursor.close()
        connection.close()

# Creating threads to simulate concurrent student registrations

thread1 = threading.Thread(target=register_student, args=("Mira", "mira@example.com", 22))
thread2 = threading.Thread(target=register_student, args=("Bob", "bob@example.com", 20))

# Starting the threads

thread1.start()
thread2.start()


# Wait for both threads to finish

thread1.join()
thread2.join()

Transaction committed successfully for Mira!

Transaction committed successfully for Bob!

